In [1]:
# Read the input from the file and split it into rows of integers
with open('./inputs/1.txt', 'r') as file:
    rows = [list(map(int, line.split())) for line in file.read().splitlines()]

# Transpose the rows to columns
columns = list(map(list, zip(*rows)))

# Sort each column
for column in columns:
    column.sort()

# Calculate the sum of absolute differences between corresponding elements in the columns
total_abs_difference = sum(abs(x - y) for x, y in zip(*columns))
print("Sum of absolute differences:", total_abs_difference)

# Separate the columns back into individual lists
left_column, right_column = map(list, zip(*rows))

# Calculate the weighted sum of the left column using counts from the right column
weighted_sum = sum(value * right_column.count(value) for value in left_column)
print("Weighted sum:", weighted_sum)


Sum of absolute differences: 2176849
Weighted sum: 23384288


In [2]:
def is_safe_sequence(levels):
    # Calculate differences between consecutive levels
    differences = [x - y for x, y in zip(levels, levels[1:])]
    # Check if all differences are within the range 1 to 3 or -1 to -3
    return all(1 <= diff <= 3 for diff in differences) or all(-1 >= diff >= -3 for diff in differences)

# Part 1: Count sequences that are safe
safe_sequence_count = 0
with open('./inputs/2.txt', 'r') as file:
    for report in file:
        levels = list(map(int, report.split()))
        if is_safe_sequence(levels):
            safe_sequence_count += 1
print("Safe sequence count (Part 1):", safe_sequence_count)

# Part 2: Count sequences that can be made safe by removing one element
modifiable_safe_sequence_count = 0
with open('./inputs/2.txt', 'r') as file:
    for report in file:
        levels = list(map(int, report.split()))
        # Check if removing any single element results in a safe sequence
        if any(is_safe_sequence(levels[:index] + levels[index + 1:]) for index in range(len(levels))):
            modifiable_safe_sequence_count += 1
print("Modifiable safe sequence count (Part 2):", modifiable_safe_sequence_count)

Safe sequence count (Part 1): 299
Modifiable safe sequence count (Part 2): 364


In [3]:
import re

def parse_mul_instructions(memory_input, part):
    # Regex patterns for different instructions
    mul_pattern = r'mul\((\d{1,3})\s*,\s*(\d{1,3})\)'
    do_pattern = r'do\(\)'
    dont_pattern = r'don\'*t\(\)'
    
    # Track multiplication state
    mul_enabled = True
    total_sum = 0
    
    # Find all instructions in order
    mul_matches = list(re.finditer(mul_pattern, memory_input))
    if part == 2:
        do_matches = list(re.finditer(do_pattern, memory_input))
        dont_matches = list(re.finditer(dont_pattern, memory_input))
        all_matches = sorted(
            mul_matches + do_matches + dont_matches, 
            key=lambda x: x.start()
        )
    else:
        all_matches = mul_matches
    
    # Process instructions in order
    for match in all_matches:
        if part == 2:
            if match.group() == 'do()':
                mul_enabled = True
            elif match.group() == "don't()":
                mul_enabled = False
        
        if match.group().startswith('mul(') and mul_enabled:
            x, y = match.groups()
            total_sum += int(x) * int(y)
    
    return total_sum

def main():
    # Read the input from a text file
    try:
        with open('./inputs/3.txt', 'r') as file:
            memory_input = file.read().strip()
        
        # Calculate the sum of multiplication results for Part 1
        result_part1 = parse_mul_instructions(memory_input, part=1)
        print(f"Part 1 - Sum of multiplication results: {result_part1}")
        
        # Calculate the sum of multiplication results for Part 2
        result_part2 = parse_mul_instructions(memory_input, part=2)
        print(f"Part 2 - Sum of multiplication results: {result_part2}")
    
    except FileNotFoundError:
        print("Error: 'corrupted_memory.txt' file not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    main()

Part 1 - Sum of multiplication results: 159833790
Part 2 - Sum of multiplication results: 89349241


In [ ]:
from typing import List, Tuple, Set
from dataclasses import dataclass

@dataclass(frozen=True)
class GridCoord:
    row: int
    col: int

    def offset_by(self, direction: Tuple[int, int], multiplier: int = 1) -> 'GridCoord':
        dr, dc = direction
        return GridCoord(
            self.row + (dr * multiplier),
            self.col + (dc * multiplier)
        )

# Constants
DIRECTIONS = [(dr, dc) for dr in [-1, 0, 1] for dc in [-1, 0, 1] if not (dr == dc == 0)]
VALID_MMSS_PATTERNS = {"MMSS", "MSSM", "SSMM", "SMMS"}

def read_grid(filename: str) -> List[str]:
    """
    Reads and processes a grid from a file.
    
    Args:
        filename: Path to the input file
        
    Returns:
        List of strings representing rows in the grid
    """
    try:
        with open(filename, 'r') as file:
            return [line.strip().upper() for line in file.readlines()]
    except FileNotFoundError:
        print(f"The file {filename} was not found.")
        return []
    except Exception as e:
        print(f"An error occurred: {e}")
        return []

def is_valid_position(pos: GridCoord, grid: List[str]) -> bool:
    """Check if a position is within grid bounds."""
    return 0 <= pos.row < len(grid) and 0 <= pos.col < len(grid[0])

def count_xmas(grid: List[str]) -> int:
    """
    Counts occurrences of 'XMAS' in all directions.
    
    Searches for 'XMAS' patterns starting from each 'X' in the grid,
    checking all 8 possible directions.
    """
    count = 0
    
    for r in range(len(grid)):
        for c in range(len(grid[0])):
            if grid[r][c] != "X":
                continue
                
            current_pos = GridCoord(r, c)
            for direction in DIRECTIONS:
                # Check if all positions for 'MAS' are valid
                positions = [current_pos.offset_by(direction, i) for i in range(1, 4)]
                if not all(is_valid_position(pos, grid) for pos in positions):
                    continue
                
                # Check if letters match 'MAS'
                if (grid[positions[0].row][positions[0].col] == "M" and
                    grid[positions[1].row][positions[1].col] == "A" and
                    grid[positions[2].row][positions[2].col] == "S"):
                    count += 1
                    
    return count

def count_mmss(grid: List[str]) -> int:
    """
    Counts occurrences of valid MMSS patterns around each 'A'.
    
    Checks the four corners around each 'A' for valid MMSS combinations.
    """
    count = 0
    
    for r in range(1, len(grid) - 1):
        for c in range(1, len(grid[0]) - 1):
            if grid[r][c] != "A":
                continue
                
            corners = [
                grid[r - 1][c - 1],  # top-left
                grid[r - 1][c + 1],  # top-right
                grid[r + 1][c + 1],  # bottom-right
                grid[r + 1][c - 1]   # bottom-left
            ]
            if "".join(corners) in VALID_MMSS_PATTERNS:
                count += 1
                
    return count

def main() -> None:
    filename = 'inputs/4.txt'
    grid = read_grid(filename)
    
    if grid:
        xmas_count = count_xmas(grid)
        mmss_count = count_mmss(grid)
        
        print(f"XMAS Count: {xmas_count}")
        print(f"MMSS Count: {mmss_count}")
    else:
        print("No grid data available.")

if __name__ == "__main__":
    main()